<a href="https://colab.research.google.com/github/Pelmenoff/data_science/blob/main/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Завантаження даних
(x, y), _ = fashion_mnist.load_data()

# Спочатку розділяємо дані на 65% тренувальні і 35% для тестування та валідації
x_train, x_test_raw, y_train, y_test_raw = train_test_split(x, y, train_size=0.7, random_state=555)

# Тепер розділимо ці 30% на 20% для тесту і 15% для валідації
# 20% від загального числа = 20/30 частини від x_test_raw та y_test_raw
test_size = 20 / 30
x_val, x_test, y_val, y_test = train_test_split(x_test_raw, y_test_raw, test_size=test_size, random_state=555)

print("Train data size", x_train.shape)
print("Test data size", x_test.shape)
print("Validation data size", x_val.shape)

# Масштабування зображень до діапазону [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Розширення розмірності (додавання каналу)
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot кодування міток
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

Train data size (42000, 28, 28)
Test data size (12000, 28, 28)
Validation data size (6000, 28, 28)


In [3]:
model = Sequential()

# Перший згортковий шар
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))

# Другий згортковий шар
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Третій згортковий шар
model.add(Conv2D(128, (3, 3), activation='relu'))

# Перетворення даних в одновимірний вектор
model.add(Flatten())

# Перший повнозв'язний шар
model.add(Dense(128, activation='relu'))

# Вихідний шар
model.add(Dense(10, activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 1152)              0

In [5]:
# Компільовання моделі
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Тренування моделі
history = model.fit(x_train, y_train, batch_size=128,
                    epochs=25, validation_data=(x_val, y_val))

Epoch 1/25
329/329 [==============================] - 42s 124ms/step - loss: 0.6462 - accuracy: 0.7641 - val_loss: 0.4264 - val_accuracy: 0.8433
Epoch 2/25
329/329 [==============================] - 39s 119ms/step - loss: 0.3900 - accuracy: 0.8583 - val_loss: 0.3843 - val_accuracy: 0.8593
Epoch 3/25
329/329 [==============================] - 41s 124ms/step - loss: 0.3310 - accuracy: 0.8794 - val_loss: 0.3464 - val_accuracy: 0.8722
Epoch 4/25
329/329 [==============================] - 42s 128ms/step - loss: 0.2974 - accuracy: 0.8918 - val_loss: 0.2868 - val_accuracy: 0.8957
Epoch 5/25
329/329 [==============================] - 38s 116ms/step - loss: 0.2734 - accuracy: 0.8986 - val_loss: 0.2700 - val_accuracy: 0.8988
Epoch 6/25
329/329 [==============================] - 40s 120ms/step - loss: 0.2534 - accuracy: 0.9069 - val_loss: 0.2907 - val_accuracy: 0.8970
Epoch 7/25
329/329 [==============================] - 40s 120ms/step - loss: 0.2279 - accuracy: 0.9154 - val_loss: 0.2602 - val_ac

In [8]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, accuracy: {test_acc}')

375/375 [==============================] - 4s 10ms/step - loss: 0.4027 - accuracy: 0.9147
Test loss: 0.40268024802207947, accuracy: 0.9146666526794434
